In [1]:
import face_recognition
import cv2
import numpy as np
import os
import openpyxl
import datetime

In [2]:
image_train=[]
classes=[]

def load_images(folder):
    for person_folder in os.listdir(folder):
        person_path = os.path.join(folder, person_folder)
        if os.path.isdir(person_path):
             for filename in os.listdir(person_path):
                path = os.path.join(person_path, filename)
                image_train.append(face_recognition.load_image_file(path))
                    
                classes.append(os.path.splitext(filename)[0])
    return image_train


In [3]:
print(image_train)

[]


In [4]:
load_images("Computer Science Department photos")

[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[107, 151, 198],
         [104, 148, 197],
         [103, 146, 197],
         ...,
         [ 81, 128, 184],
         [ 84, 128, 189],
         [ 94, 138, 201]],
 
        [[108, 152, 199],
         [106, 150, 197],
         [107, 151, 200],
         ...,
         [ 89, 136, 192],
         [ 99, 145, 205],
         [109, 155, 217]],
 
        [[105, 152, 196],
         [104, 151, 197],
         [108, 152, 201],
         ...,
         [ 94, 141, 197],
  

In [5]:
def findEncodings(images):
    encodeList = []

    for img in images:
        img = cv2.cvtColor(src=img, code=cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(face_image=img)[0]
        encodeList.append(encode)
    return encodeList
    

In [6]:
findEncodings(image_train)

[array([-0.12401451,  0.06069894,  0.04328865, -0.00651008, -0.09620833,
        -0.02331063, -0.02577181, -0.00810739,  0.12187534, -0.08965754,
         0.15001534, -0.02929777, -0.15348382, -0.08224291,  0.045017  ,
         0.10775146, -0.17233863, -0.11095245, -0.0855275 , -0.05185886,
         0.038714  ,  0.00287375, -0.01076477,  0.04142931, -0.16162068,
        -0.37766579, -0.08126527, -0.09799813, -0.01297078, -0.10582923,
        -0.00317766,  0.08223765, -0.12924266, -0.00094402, -0.06317879,
         0.1052508 , -0.09063371, -0.07782392,  0.21258926, -0.00247913,
        -0.16262013, -0.08419856,  0.0456434 ,  0.29641587,  0.22952975,
         0.01001398,  0.00156528,  0.04497319,  0.13554962, -0.27214047,
         0.02640588,  0.13803323,  0.03682689,  0.09709951,  0.10465892,
        -0.08260746,  0.02984899,  0.1388987 , -0.20568426,  0.04871165,
        -0.06157207, -0.13749427, -0.02340269, -0.06101526,  0.25292793,
         0.16571695, -0.08122067, -0.07366426,  0.1

In [7]:
def takeAttendance(attendance):

    global index

    # Loading the excel file where we will save the attendance. 
    xlsx_path = "attendance.xlsx"
    wb = openpyxl.load_workbook(filename=xlsx_path)

    # Check if the sheet for the current date exists
    sheet_name = str(datetime.datetime.now()).split()[0]
    sheet = wb[sheet_name] if sheet_name in wb.sheetnames else None

    # If the sheet does not exist, create a new one
    if sheet is None:
        sheet = wb.create_sheet(title=sheet_name)

        # Add headers
        sheet.cell(row=1, column=1, value="Name")
        sheet.cell(row=1, column=2, value="Date")
        sheet.cell(row=1, column=3, value="Time")

    # Find the last row with data
    index = sheet.max_row + 1 if sheet.max_row > 1 else 2

    for name in attendance:
        attend_date, attend_time = str(datetime.datetime.now()).split()
        sheet.cell(row=index, column=1, value=name)
        sheet.cell(row=index, column=2, value=attend_date)
        sheet.cell(row=index, column=3, value=attend_time[:8])
        index += 1

    # Save the workbook
    wb.save(filename=xlsx_path)


In [9]:
from pathlib import Path
import pandas as pd
import datetime

today_directory=Path('Attendence_img/'+str(datetime.datetime.now()).split()[0])

if today_directory.exists:
    None
else:
    os.mkdir(today_directory)


# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Create arrays of known face encodings and their names
known_face_encodings = findEncodings(image_train)
known_face_names = classes

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
attendence=[]
process_this_frame = True



while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
            
            
            #created detected face list
            if (name in attendence) or (name =="Unknown"):
                None
            else: 
                attendence.append(name)

    process_this_frame = not process_this_frame
    
           # Create a DataFrame and save to Excel
    df = pd.DataFrame(attendence)    
    df.to_excel('attendance.xlsx', index=False)


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
            

        # Draw a box around the face
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
    cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#insert attendance in the attendance sheet 
takeAttendance(attendence)


# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

NameError: name 'left' is not defined